<img src="https://s8.hostingkartinok.com/uploads/images/2018/08/308b49fcfbc619d629fe4604bceb67ac.jpg" width=500, height=450>
<h3 style="text-align: center;"><b>Физтех-Школа Прикладной математики и информатики (ФПМИ) МФТИ</b></h3>

---

***Some parts of the notebook are almost the copy of [ mmta-team course](https://github.com/mmta-team/mmta_fall_2020). Special thanks to mmta-team for making them publicly available. [Original notebook](https://github.com/mmta-team/mmta_fall_2020/blob/master/tasks/01_word_embeddings/task_word_embeddings.ipynb).***

<b> Прочитайте семинар, пожалуйста, для успешного выполнения домашнего задания. В конце ноутка напишите свой вывод. Работа без вывода оценивается ниже.

## Задача поиска схожих по смыслу предложений

Мы будем ранжировать вопросы [StackOverflow](https://stackoverflow.com) на основе семантического векторного представления

До этого в курсе не было речи про задачу ранжировния, поэтому введем математическую формулировку

## Задача ранжирования(Learning to Rank)

* $X$ - множество объектов
* $X^l = \{x_1, x_2, ..., x_l\}$ - обучающая выборка
<br>На обучающей выборке задан порядок между некоторыми элементами, то есть нам известно, что некий объект выборки более релевантный для нас, чем другой:
* $i \prec j$ - порядок пары индексов объектов на выборке $X^l$ c индексами $i$ и $j$
### Задача:
построить ранжирующую функцию $a$ : $X \rightarrow R$ такую, что
$$i \prec j \Rightarrow a(x_i) < a(x_j)$$

<img src="https://d25skit2l41vkl.cloudfront.net/wp-content/uploads/2016/12/Featured-Image.jpg" width=500, height=450>

### Embeddings

Будем использовать предобученные векторные представления слов на постах Stack Overflow.<br>
[A word2vec model trained on Stack Overflow posts](https://github.com/vefstathiou/SO_word2vec)

In [ ]:
!wget https://zenodo.org/record/1199620/files/SO_vectors_200.bin?download=1

--2024-10-09 18:33:53--  https://zenodo.org/record/1199620/files/SO_vectors_200.bin?download=1
Resolving zenodo.org (zenodo.org)... 188.184.98.238, 188.184.103.159, 188.185.79.172, ...
Connecting to zenodo.org (zenodo.org)|188.184.98.238|:443... connected.
HTTP request sent, awaiting response... 301 MOVED PERMANENTLY
Location: /records/1199620/files/SO_vectors_200.bin [following]
--2024-10-09 18:33:54--  https://zenodo.org/records/1199620/files/SO_vectors_200.bin
Reusing existing connection to zenodo.org:443.
HTTP request sent, awaiting response... 200 OK
Length: 1453905423 (1.4G) [application/octet-stream]
Saving to: ‘SO_vectors_200.bin?download=1’

SO_vectors_200.bin? 100%[===================>]   1.35G  22.1MB/s    in 53s     

2024-10-09 18:34:47 (26.3 MB/s) - ‘SO_vectors_200.bin?download=1’ saved [1453905423/1453905423]



In [ ]:
from gensim.models.keyedvectors import KeyedVectors
wv_embeddings = KeyedVectors.load_word2vec_format("SO_vectors_200.bin?download=1", binary=True)

#### Как пользоваться этими векторами?

Посмотрим на примере одного слова, что из себя представляет embedding

In [ ]:
word = 'dog'
if word in wv_embeddings:
    print(wv_embeddings[word].dtype, wv_embeddings[word].shape)

float32 (200,)


In [ ]:
print(f"Num of words: {len(wv_embeddings.index_to_key)}")

Num of words: 1787145


Найдем наиболее близкие слова к слову `dog`:

#### Вопрос 1:
* Входит ли слов `cat` топ-5 близких слов к слову `dog`? Какое место?

In [ ]:
# method most_simmilar
wv_embeddings.most_similar(positive=word)

[('animal', 0.8564180135726929),
 ('dogs', 0.7880866527557373),
 ('mammal', 0.7623804211616516),
 ('cats', 0.7621253728866577),
 ('animals', 0.760793924331665),
 ('feline', 0.7392398715019226),
 ('bird', 0.7315488457679749),
 ('animal1', 0.7219215631484985),
 ('doggy', 0.7213349342346191),
 ('labrador', 0.7209131717681885)]

**Ответ:** входит однокоренное слово `cats` на четвертом месте

### Векторные представления текста

Перейдем от векторных представлений отдельных слов к векторным представлениям вопросов, как к **среднему** векторов всех слов в вопросе. Если для какого-то слова нет предобученного вектора, то его нужно пропустить. Если вопрос не содержит ни одного известного слова, то нужно вернуть нулевой вектор.

In [ ]:
import numpy as np
import re
# you can use your tokenizer
# for example, from nltk.tokenize import WordPunctTokenizer
class MyTokenizer:
    def __init__(self):
        pass
    def tokenize(self, text):
        return re.findall('\w+', text)

from nltk.tokenize import WordPunctTokenizer
tokenizer = WordPunctTokenizer()

In [ ]:
def question_to_vec(question, embeddings = wv_embeddings, tokenizer = WordPunctTokenizer(), dim=200):
    """
        question: строка
        embeddings: наше векторное представление
        dim: размер любого вектора в нашем представлении

        return: векторное представление для вопроса
    """
    tokens = tokenizer.tokenize(question) # берем токены
    vector = [] # сюда будем сохранять векторы слов, потом внутри усредним
    for token in tokens:
      if token in embeddings:
        vector.append(embeddings[token][:dim]) # второй слайс нужен, чтобы извлечь нужное количество элементов вектора по параметру dim
      else: # обрабатываем исключения
        pass
    if not vector:
      return np.zeros(dim)
    else:
      return np.mean(vector, axis=0)


Потыкаемся в примеры:

In [ ]:
q1 = 'Considering the rapid advancements in artificial intelligence and machine learning technologies, how do you foresee these developments impacting the future job market, particularly in terms of employment opportunities, skill requirements, and potential shifts in various industries over the next decade?'
question_to_vec(q1)

array([ 0.35106915,  0.10771632, -0.25482503,  0.6402642 , -0.7235314 ,
       -0.11992723,  0.97349256, -0.43888173,  0.31916288, -0.5036137 ,
       -0.8691242 ,  0.65037155, -0.24344051, -0.19825996,  0.73817706,
        0.790657  , -0.7558123 ,  0.8085703 , -0.7433328 , -0.11717838,
       -0.5411776 ,  0.63282347,  0.620498  , -0.4549313 , -0.36077005,
       -0.31737533, -0.04515034, -0.2671843 , -0.27800536, -0.2806059 ,
       -0.92948633, -0.44331083, -1.5966698 , -0.5493556 ,  1.5000752 ,
        1.330663  , -0.02201211,  0.23166214, -0.20218237, -0.3240934 ,
       -0.74329567,  0.3333603 ,  0.45496723, -0.17825447, -0.14723137,
        0.1360338 , -1.09883   ,  0.41014412, -0.47604734, -0.49919018,
        0.8716211 ,  0.06063569, -1.5021958 ,  0.3475379 , -0.62868047,
        1.5042353 ,  0.28912362, -0.5611629 ,  0.73560137, -0.21212299,
        0.29926217,  0.32037067, -0.44478267,  0.06252588,  1.0870659 ,
        0.5579184 , -0.18170266,  0.15763849, -0.80350876, -0.10

In [ ]:
q2 = 'Abc Def Ghi'
question_to_vec(q2)

array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.])

In [ ]:
q3 = 'In the context of global health challenges, such as the recent COVID-19 pandemic, what are the key lessons that public health systems around the world can learn to improve their preparedness, response, and recovery efforts for future pandemics or similar health crises?'
question_to_vec(q3)

array([ 5.54454327e-01,  2.52267987e-01, -4.14460629e-01,  3.80052179e-01,
        1.81033343e-01,  3.89086545e-01,  1.23848200e+00, -2.77398109e-01,
        1.41722456e-01,  2.02618748e-01, -5.18557489e-01,  5.43784082e-01,
       -9.64684412e-02,  2.84182429e-01,  9.00031254e-02,  8.95340681e-01,
       -5.19919442e-03,  2.70409375e-01, -5.95360756e-01, -6.57279566e-02,
       -2.79843211e-01, -9.97699350e-02,  2.41742246e-02, -7.60769397e-02,
        1.39587428e-02, -3.35225940e-01, -1.29079435e-03,  9.05251782e-03,
       -9.95390341e-02, -1.43365800e-01, -6.55798256e-01, -8.63194168e-02,
       -4.84080046e-01, -3.93042475e-01,  1.84040055e-01,  1.27627826e+00,
       -3.99433017e-01,  5.48189059e-02,  6.99700713e-02, -6.95514008e-02,
       -3.54457021e-01,  6.05109572e-01,  9.13604736e-01, -3.05639833e-01,
       -5.83856106e-01,  4.41241384e-01, -1.23039007e+00,  6.71413660e-01,
       -5.32152593e-01,  1.46558896e-01,  7.57376850e-01,  1.37092307e-01,
       -1.50680149e+00,  

In [ ]:
q4 = 'In the context of global health challenges, such as the recent pandemic, what are the key lessons that public health systems around the world can learn to improve their preparedness, response, and recovery efforts for future pandemics or similar health crises?'
question_to_vec(q4)

array([ 0.580512  ,  0.2954305 , -0.4434848 ,  0.34466162,  0.26894078,
        0.30798474,  1.3464453 , -0.44868562,  0.1437147 ,  0.3428303 ,
       -0.37814334,  0.6143836 , -0.19648209,  0.28038165,  0.10208936,
        0.98152477,  0.0907975 ,  0.44205377, -0.5470578 ,  0.03565048,
       -0.23028411,  0.01215417,  0.08503368, -0.27234292, -0.03525097,
       -0.3333631 ,  0.14163913, -0.00342428, -0.05801354, -0.0669136 ,
       -0.73205954,  0.0416496 , -0.4078753 , -0.37517014,  0.4018988 ,
        1.228144  , -0.4475004 ,  0.08282857,  0.05979627, -0.1323223 ,
       -0.56664383,  0.5869669 ,  1.0329586 , -0.17686975, -0.5547079 ,
        0.47291797, -1.254138  ,  0.6400787 , -0.6218733 ,  0.18006738,
        0.67730856,  0.17088182, -1.5256855 ,  0.44393674, -0.41434   ,
        0.998398  ,  0.45970893, -0.19150259,  0.45458764, -0.29123947,
        0.8595214 , -0.2594019 ,  0.586548  ,  0.7175766 ,  0.69061655,
        0.20339638,  0.10760916,  0.23499961, -0.09482593,  0.38

`То есть этот эмбеддинг даже считает COVID-19 за слово. Как же он хорош`

Все же посмотрим на то, как он обрабатывает исключения:

In [ ]:
q5 = 'dog'
q6 = 'dog Abrabra'
question_to_vec(q5) == question_to_vec(q6) #так как второе слово бессмысленное

array([ True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,

Теперь у нас есть метод для создания векторного представления любого предложения.

#### Вопрос 2:
* Какая третья(с индексом 2) компонента вектора предложения `I love neural networks` (округлите до 2 знаков после запятой)?

In [ ]:
q = 'I love neural networks'
round(question_to_vec(q)[2], 2)

-1.29

### Оценка близости текстов

Представим, что мы используем идеальные векторные представления слов. Тогда косинусное расстояние между дублирующими предложениями должно быть меньше, чем между случайно взятыми предложениями.

Сгенерируем для каждого из $N$ вопросов $R$ случайных отрицательных примеров и примешаем к ним также настоящие дубликаты. Для каждого вопроса будем ранжировать с помощью нашей модели $R + 1$ примеров и смотреть на позицию дубликата. Мы хотим, чтобы дубликат был первым в ранжированном списке.

#### Hits@K
Первой простой метрикой будет количество корректных попаданий для какого-то $K$:
$$ \text{Hits@K} = \frac{1}{N}\sum_{i=1}^N \, [rank\_q_i^{'} \le K],$$
* $\begin{equation*}
[x < 0 ] \equiv
 \begin{cases}
   1, &x < 0\\
   0, &x \geq 0
 \end{cases}
\end{equation*}$ - индикаторная функция
* $q_i$ - $i$-ый вопрос
* $q_i^{'}$ - его дубликат
* $rank\_q_i^{'}$ - позиция дубликата в ранжированном списке ближайших предложений для вопроса $q_i$.

#### DCG@K
Второй метрикой будет упрощенная DCG метрика, учитывающая порядок элементов в списке путем домножения релевантности элемента на вес равный обратному логарифму номера позиции::
$$ \text{DCG@K} = \frac{1}{N} \sum_{i=1}^N\frac{1}{\log_2(1+rank\_q_i^{'})}\cdot[rank\_q_i^{'} \le K],$$
С такой метрикой модель штрафуется за большой ранк корректного ответа

#### Вопрос 3:
* Максимум `Hits@47 - DCG@1`?

In [ ]:
def hits_count(dup_ranks, k):
    """
        dup_ranks: list индексов дубликатов
        result: вернуть  Hits@k
    """
    hits_value = sum(1 for rank in dup_ranks if rank <= k) / len(dup_ranks)
    return hits_value
def dcg_score(dup_ranks, k):
    """
        dup_ranks: list индексов дубликатов
        result: вернуть DCG@k
    """
    import math
    dcg_value = sum((1 / (math.log2(1 + rank))) for rank in dup_ranks if rank <= k) / len(dup_ranks)
    return dcg_value
dup_ranks_list = list(range(1, 47))
print([hits_count(dup_ranks_list, 47) - dcg_score(dup_ranks_list, 1)])


[0.9782608695652174]


<img src='https://hsto.org/files/1c5/edf/dee/1c5edfdeebce4b71a86bdf986d9f88f2.jpg' width=400, height=200>

#### Пример оценок

Вычислим описанные выше метрики для игрушечного примера.
Пусть
* $N = 1$, $R = 3$
* <font color='green'>"Что такое python?"</font> - вопрос $q_1$
* <font color='red'>"Что такое язык python?"</font> - его дубликат $q_i^{'}$

Пусть модель выдала следующий ранжированный список кандидатов:

1. "Как изучить с++?"
2. <font color='red'>"Что такое язык python?"</font>
3. "Хочу учить Java"
4. "Не понимаю Tensorflow"

$\Rightarrow rank\_q_i^{'} = 2$

Вычислим метрику *Hits@K* для *K = 1, 4*:

- [K = 1] $\text{Hits@1} =  [rank\_q_i^{'} \le 1)] = 0$
- [K = 4] $\text{Hits@4} =  [rank\_q_i^{'} \le 4] = 1$

Вычислим метрику *DCG@K* для *K = 1, 4*:
- [K = 1] $\text{DCG@1} = \frac{1}{\log_2(1+2)}\cdot[2 \le 1] = 0$
- [K = 4] $\text{DCG@4} = \frac{1}{\log_2(1+2)}\cdot[2 \le 4] = \frac{1}{\log_2{3}}$

#### Вопрос 4:
* Вычислите `DCG@10`, если $rank\_q_i^{'} = 9$(округлите до одного знака после запятой)

**Решение:**

$$ \text{DCG@10} = \frac{1}{N} \sum_{i=1}^N\frac{1}{\log_2(1+rank\_q_i^{'})}\cdot[rank\_q_i^{'} \le K] = \frac{1}{N} \sum_{i=1}^N\frac{1}{\log_210} = \frac{1}{\log_210} \approx 0.3$$

### HITS\_COUNT и DCG\_SCORE

Каждая функция имеет два аргумента: $dup\_ranks$ и $k$. $dup\_ranks$ является списком, который содержит рейтинги дубликатов(их позиции в ранжированном списке). Например, $dup\_ranks = [2]$ для примера, описанного выше.

In [ ]:
def hits_count(dup_ranks, k):
    """
        dup_ranks: list индексов дубликатов
        result: вернуть  Hits@k
    """
    hits_value = sum(1 for rank in dup_ranks if rank <= k) / len(dup_ranks)
    return hits_value

In [ ]:
def dcg_score(dup_ranks, k):
    """
        dup_ranks: list индексов дубликатов
        result: вернуть DCG@k
    """
    import math
    dcg_value = sum((1 / (math.log2(1 + rank))) for rank in dup_ranks if rank <= k) / len(dup_ranks)
    return dcg_value

Протестируем функции. Пусть $N = 1$, то есть один эксперимент. Будем искать копию вопроса и оценивать метрики.

In [ ]:
import pandas as pd

In [ ]:
copy_answers = ["How does the catch keyword determine the type of exception that was thrown",]

# наги кандидаты
candidates_ranking = [["How Can I Make These Links Rotate in PHP",
                       "How does the catch keyword determine the type of exception that was thrown",
                       "NSLog array description not memory address",
                       "PECL_HTTP not recognised php ubuntu"],]
# dup_ranks — позиции наших копий, так как эксперимент один, то этот массив длины 1
dup_ranks = [2]

# вычисляем метрику для разных k
print('Ваш ответ HIT:', [hits_count(dup_ranks, k) for k in range(1, 5)])
print('Ваш ответ DCG:', [round(dcg_score(dup_ranks, k), 5) for k in range(1, 5)])

Ваш ответ HIT: [0.0, 1.0, 1.0, 1.0]
Ваш ответ DCG: [0.0, 0.63093, 0.63093, 0.63093]


У вас должно получиться

In [ ]:
# correct_answers - метрика для разных k
correct_answers = pd.DataFrame([[0, 1, 1, 1], [0, 1 / (np.log2(3)), 1 / (np.log2(3)), 1 / (np.log2(3))]],
                               index=['HITS', 'DCG'], columns=range(1,5))
correct_answers

,1,2,3,4
HITS,0,1.00000,1.00000,1.00000
DCG,0,0.63093,0.63093,0.63093


In [ ]:
'''
Кажется, сошлось
'''

### Данные
[arxiv link](https://drive.google.com/file/d/1QqT4D0EoqJTy7v9VrNCYD-m964XZFR7_/edit)

`train.tsv` - выборка для обучения.<br> В каждой строке через табуляцию записаны: **<вопрос>, <похожий вопрос>**

`validation.tsv` - тестовая выборка.<br> В каждой строке через табуляцию записаны: **<вопрос>, <похожий вопрос>, <отрицательный пример 1>, <отрицательный пример 2>, ...**

In [ ]:
!gdown 1QqT4D0EoqJTy7v9VrNCYD-m964XZFR7_

Downloading...
From (original): https://drive.google.com/uc?id=1QqT4D0EoqJTy7v9VrNCYD-m964XZFR7_
From (redirected): https://drive.google.com/uc?id=1QqT4D0EoqJTy7v9VrNCYD-m964XZFR7_&confirm=t&uuid=5f66e243-a27f-466e-ba23-dd1ce2221493
To: /content/stackoverflow_similar_questions.zip
100% 131M/131M [00:00<00:00, 150MB/s]


In [ ]:
!unzip stackoverflow_similar_questions.zip

Archive:  stackoverflow_similar_questions.zip
   creating: data/
  inflating: data/.DS_Store          
   creating: __MACOSX/
   creating: __MACOSX/data/
  inflating: __MACOSX/data/._.DS_Store  
  inflating: data/train.tsv          
  inflating: data/validation.tsv     


Считайте данные.

In [ ]:
def read_corpus(filename):
    data = []
    for line in open(filename, encoding='utf-8'):
        fields = line.strip().split('\t')
        data.append(fields)
    return data

Нам понадобиться только файл validation.

In [ ]:
validation_data = read_corpus('./data/validation.tsv')

Кол-во строк

In [ ]:
len(validation_data)

3760

Размер нескольких первых строк

In [ ]:
for i in range(5):
    print(i + 1, len(validation_data[i]))

1 1001
2 1001
3 1001
4 1001
5 1001


### Ранжирование без обучения

Реализуйте функцию ранжирования кандидатов на основе косинусного расстояния. Функция должна по списку кандидатов вернуть отсортированный список пар (позиция в исходном списке кандидатов, кандидат). При этом позиция кандидата в полученном списке является его рейтингом (первый - лучший). Например, если исходный список кандидатов был [a, b, c], и самый похожий на исходный вопрос среди них - c, затем a, и в конце b, то функция должна вернуть список **[(2, c), (0, a), (1, b)]**.

In [ ]:
from sklearn.metrics.pairwise import cosine_similarity
from copy import deepcopy

In [ ]:
def rank_candidates(question, candidates, embeddings, tokenizer = WordPunctTokenizer(), dim=200):
    """
        question: строка
        candidates: массив строк(кандидатов) [a, b, c]
        result: пары (начальная позиция, кандидат) [(2, c), (0, a), (1, b)]
    """
    question_embedding = question_to_vec(question, embeddings, tokenizer, dim).reshape(1, -1)
    candidate_embeddings = []
    for candidate in candidates:
        candidate_embeddings.append(question_to_vec(candidate, embeddings, tokenizer, dim).reshape(1, -1))
    candidate_embeddings_matrix = np.vstack(candidate_embeddings)
    similarities = cosine_similarity(question_embedding, candidate_embeddings_matrix).flatten()
    #print(similarities) использовал при отладке
    indexed_candidates = list(enumerate(candidates))
    sorted_candidates = sorted(indexed_candidates, key=lambda x: similarities[x[0]], reverse=True)

    return sorted_candidates



Протестируйте работу функции на примерах ниже. Пусть $N=2$, то есть два эксперимента

In [ ]:
questions = ['converting string to list', 'Sending array via Ajax fails']

candidates = [['Convert Google results object (pure js) to Python object', # первый эксперимент
               'C# create cookie from string and send it',
               'How to use jQuery AJAX for an outside domain?'], \

              ['Getting all list items of an unordered list in PHP',      # второй эксперимент
               'WPF- How to update the changes in list item of a list',
               'select2 not displaying search results']]

In [ ]:
for question, q_candidates in zip(questions, candidates):
        ranks = rank_candidates(question, q_candidates, wv_embeddings, tokenizer)
        print(ranks)


[(1, 'C# create cookie from string and send it'), (0, 'Convert Google results object (pure js) to Python object'), (2, 'How to use jQuery AJAX for an outside domain?')]
[(1, 'WPF- How to update the changes in list item of a list'), (0, 'Getting all list items of an unordered list in PHP'), (2, 'select2 not displaying search results')]


Для первого экперимента вы можете полностью сравнить ваши ответы и правильные ответы. Но для второго эксперимента два ответа на кандидаты будут <b>скрыты</b>(*)

In [ ]:
# должно вывести
results = [[(1, 'C# create cookie from string and send it'),
            (0, 'Convert Google results object (pure js) to Python object'),
            (2, 'How to use jQuery AJAX for an outside domain?')],
           [(*, 'Getting all list items of an unordered list in PHP'), #скрыт
            (*, 'select2 not displaying search results'), #скрыт
            (*, 'WPF- How to update the changes in list item of a list')]] #скрыт

Последовательность начальных индексов вы должны получить `для эксперимента 1`  1, 0, 2.

#### Вопрос 5:
* Какую последовательность начальных индексов вы получили `для эксперимента 2`(перечисление без запятой и пробелов, например, `102` для первого эксперимента?

**Ответ: 102**

Теперь мы можем оценить качество нашего метода. Запустите следующие два блока кода для получения результата. Обратите внимание, что вычисление расстояния между векторами занимает некоторое время (примерно 10 минут). Можете взять для validation 1000 примеров.

In [ ]:
from tqdm.notebook import tqdm

In [46]:
wv_ranking = []
max_validation_examples = 1000
for i, line in enumerate(tqdm(validation_data)):
    if i == max_validation_examples:
        break
    q, *ex = line
    ranks = rank_candidates(q, ex, wv_embeddings, tokenizer)
    wv_ranking.append([r[0] for r in ranks].index(0) + 1)

  0%|          | 0/3760 [00:00<?, ?it/s]

In [47]:
for k in tqdm([1, 5, 10, 100, 500, 1000]):
    print("DCG@%4d: %.3f | Hits@%4d: %.3f" % (k, dcg_score(wv_ranking, k), k, hits_count(wv_ranking, k)))

  0%|          | 0/6 [00:00<?, ?it/s]

DCG@   1: 0.276 | Hits@   1: 0.276
DCG@   5: 0.332 | Hits@   5: 0.386
DCG@  10: 0.349 | Hits@  10: 0.438
DCG@ 100: 0.395 | Hits@ 100: 0.671
DCG@ 500: 0.421 | Hits@ 500: 0.875
DCG@1000: 0.434 | Hits@1000: 1.000


In [ ]:
'''
выглядит не очень плохо
'''


### Эмбеддинги, обученные на корпусе похожих вопросов

In [48]:
train_data = read_corpus('./data/train.tsv')

Улучшите качество модели.<br>Склеим вопросы в пары и обучим на них модель Word2Vec из gensim. Выберите размер window. Объясните свой выбор.

In [81]:
td =  [tokenizer.tokenize(" ".join(pair)) for pair in train_data]

In [82]:
td[3]

['How',
 'to',
 'insert',
 'CookieCollection',
 'to',
 'CookieContainer',
 '?',
 'C',
 '#',
 'create',
 'cookie',
 'from',
 'string',
 'and',
 'send',
 'it']

In [87]:
from gensim.models import Word2Vec
embeddings_trained = Word2Vec(td, # data for model to train on
                 vector_size=200,                 # embedding vector size
                 min_count=5,             # consider words that occured at least 5 times
                 window=7).wv #выбрал 7, по три слова до и по три после - на случай служебных частей речи

In [88]:
wv_ranking = []
max_validation_examples = 1000
for i, line in enumerate(tqdm(validation_data)):
    if i == max_validation_examples:
        break
    q, *ex = line
    ranks = rank_candidates(q, ex, embeddings_trained, tokenizer)
    wv_ranking.append([r[0] for r in ranks].index(0) + 1)

  0%|          | 0/3760 [00:00<?, ?it/s]

In [89]:
for k in tqdm([1, 5, 10, 100, 500, 1000]):
    print("DCG@%4d: %.3f | Hits@%4d: %.3f" % (k, dcg_score(wv_ranking, k), k, hits_count(wv_ranking, k)))

  0%|          | 0/6 [00:00<?, ?it/s]

DCG@   1: 0.266 | Hits@   1: 0.266
DCG@   5: 0.336 | Hits@   5: 0.397
DCG@  10: 0.356 | Hits@  10: 0.458
DCG@ 100: 0.410 | Hits@ 100: 0.727
DCG@ 500: 0.436 | Hits@ 500: 0.932
DCG@1000: 0.443 | Hits@1000: 1.000


### Замечание:
Решить эту задачу с помощью обучения полноценной нейронной сети будет вам предложено, как часть задания в одной из домашних работ по теме "Диалоговые системы".

Напишите свой вывод о полученных результатах.
* Какой принцип токенизации даёт качество лучше и почему?
* Помогает ли нормализация слов?
* Какие эмбеддинги лучше справляются с задачей и почему?
* Почему получилось плохое качество решения задачи?
* Предложите свой подход к решению задачи.

## Вывод:




*   Я пользовался только преднастроенной токенизацией, но мы видим, что она справляется неплохо. Думаю, реалистично написать кастомную с лучшим качеством, но этим займутся потомки
*   В общем случае да
*   Преднастроенный Word2Vec справился хорошо, потому что он умнее, чем то, что я могу выдать на данном уровне развития, а **идея использовать усреднение слов странная**. Почему предлоги, к примеру, также важны для вопроса, как и вопросительные слова или глаголы? Не интуитивно(((
*   Опять же, идея использовать усреднение странная.
* Использовать более продвинутую архитектуру. Можно попробовать и улучшить текущий конфиг (доучить модели, поискать более оптимальные параметры, перебрать другие эмбеддинги)



